In [7]:
import os
import pandas as pd
#import numpy as np
#import matplotlib.pyplot as plt
#import seaborn as sns
import geopandas
import folium
import branca.colormap as colormap

### Load data from filenames
#parent_folder = os.getcwd() + '/'
parent_folder = '/Users/Karen/OneDrive/GitHub/ctc21-doric-tiles/'
code_folder = parent_folder + 'code/'
data_folder =  parent_folder + 'data/'

### Main dataset
parishes = pd.read_csv(data_folder + 'CivilParish1930Lookup.csv')


### Supporting datasets
parish_area = geopandas.read_file(data_folder + 'CivilParish1930_epsg4326.shp')
### Temp filter to help loadtime
parish_area = parish_area[parish_area['name'].str.startswith('A')]
print(parish_area.head(5))

### Finding the dynamic map starting point
starting_pos = str(parish_area['geometry'][0]) \
                    .split('((')[1] \
                    .replace('))','') \
                    .split(',')[0] \
                    .split(' ') \

starting_long = starting_pos[1]
starting_lat = starting_pos[0]





      SHAPE_Leng    SHAPE_Area code C91code1 C91code2 C91code3 C91code4  \
4   41368.342275  2.932819e+07  019    05001     None     None     None   
11  58854.490736  1.135828e+08  008    07001     None     None     None   
16  39542.312713  3.620057e+07  016    51002     None     None     None   
27  30126.559506  3.092536e+07  021    05002     None     None     None   
47  49622.195601  5.113174e+07  061    45001     None     None     None   

         name                                           geometry  
4       Alloa  MULTIPOLYGON (((-3.81514 56.10242, -3.81924 56...  
11  Aberfoyle  POLYGON ((-4.39592 56.14749, -4.39647 56.14767...  
16     Airlie  POLYGON ((-3.15584 56.65699, -3.15555 56.65681...  
27       Alva  POLYGON ((-3.76332 56.14989, -3.76352 56.14973...  
47        Ayr  POLYGON ((-4.55361 55.42698, -4.55375 55.42697...  


KeyError: 0

In [ ]:
### Basic map layer
m = folium.Map( \
            location=[starting_long, starting_lat], 
            zoom_start=12,
            tiles='cartodbpositron')

### drawing datazone boundaries only
pa_style = {}
folium.GeoJson(parish_area,
                name='parishes',
               style_function=lambda x: {'fillColor': '#ff0000', 
                                         'color': '#ff0000', 
                                         'weight': 1}) \
            .add_to(m)

### Hover over tooltip and geo highlight
map_hover = folium.GeoJson(parish_area,
                    control=False,
                    style_function=lambda x: {'fillColor': '#FFFFFF', 
                                                'color':'#000000', 
                                                'fillOpacity': 0.1, 
                                                'weight': 0.1}, # invisible layer    
                    highlight_function=lambda x: {'fillColor': '#000000', 
                                                    'color':'#000000', 
                                                    'fillOpacity': 0.50, 
                                                    'weight': 0.1}, # style when hovering over                                                                                                
                    tooltip=folium.GeoJsonTooltip(fields=['name'],
                                                    aliases=['Parish'])
)

m.add_child(map_hover)

### Setting layer order and other overheads
m.keep_in_front(map_hover)

folium.LayerControl().add_to(m)

m.save(parent_folder + 'map.html')
m